In [4]:
from starter import *
from anneal import *
import math
import multiprocess as mp
import copy, random
from collections import defaultdict, Counter
import itertools
from bs4 import BeautifulSoup
import requests
tar('outputs', overwrite = True)

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('best_scores.csv')
sum(df['score'] > 200000)

50

In [6]:
tar('outputs', overwrite = True)

In [13]:
def get_all_low():
    f = open('all.txt', 'r')
    
    lines = f.readlines()
    names = []
    for line in lines:
        size, num = line.strip().split()
        names.append((size+num+'.in', size, num))
    return names
low_scores_all = get_all_low()
len(low_scores_all)

559

In [7]:

def apply_pi(G, pi):
    num_v = len(G.nodes)
    assert num_v == len(pi)
    max_p = max(pi)

    p = list(Counter(pi))
    p.sort()
    map_p = {p[i]:i for i in range(len(p))}
    for v in range(num_v):
        G.nodes[v]['team'] = int(map_p[pi[v]] + 1)
    return G  

In [19]:
solved = []
for file,target,k in (targets):
    df = pd.read_csv('best_scores.csv')
    r = df.index[df['name'] == file]
    min_cost = float(df.iloc[r]['score'])
    if min_cost <= target:
        solved.append((file, min_cost, target))
len(solved)

26

In [30]:
def write(instance, in_file, out_file, file_name, cost, k, overwrite):
    df = pd.read_csv('best_scores.csv')
    r = df.index[df['name'] == file_name]
    original = float(df.loc[r, 'score'])
    assert cost < original
    df.loc[r, 'score'] = cost
    df.loc[r, 'k'] = k
    df.to_csv('best_scores.csv', index=False)
    print(f"{str(in_file)}: cost", original, '->',score(instance))
    write_output(instance, out_file, overwrite)

In [28]:
df2 = pd.read_csv('best_scores_first.csv')


In [31]:
for file, cost_my, cost_target in targets:
    G = read_input('inputs/'+file)
    f_my = open(str(Path('outputs') / f"{file[:-len('.in')]}.out"))
    f_first = open(str(Path('outputs_firsthalf') / f"{file[:-len('.in')]}.out"))
    pi_my = json.load(f_my)
    apply_pi(G, pi_my)
    score_my = score(G)
    
    pi_first = json.load(f_first)
    apply_pi(G, pi_first)
    score_first = score(G)
    
    r = df.index[df['name'] == file]
    k = int(df.loc[r, 'k'])
    
    if score_first < score_my:
        write(G,'inputs/'+file,str(Path('outputs') / f"{file[:-len('.in')]}.out"), file, score_first,k, True)
    f_my.close()
    f_first.close()

inputs/small215.in: cost 2040.797760513179 -> 2010.797760513179
inputs/medium191.in: cost 449.5598517448982 -> 449.1689070338065
inputs/large38.in: cost 18059.82968095557 -> 15053.881984455735
inputs/large40.in: cost 18602.78135317254 -> 15060.815003314423
inputs/large203.in: cost 12351.2596504033 -> 12351.259650403294
inputs/small103.in: cost 1220.940494621729 -> 1219.2493960703473
inputs/medium177.in: cost 17327.40737527859 -> 15898.957198246593
inputs/medium224.in: cost 35346.0426970733 -> 34540.042697073266
inputs/medium228.in: cost 21451.4045262292 -> 20731.30779981041
inputs/medium250.in: cost 32082.70685496875 -> 31532.08691600961
inputs/small251.in: cost 24526.186331799625 -> 23796.992492811816
inputs/small175.in: cost 83420.49323151434 -> 81783.90918776828
inputs/small233.in: cost 5530.506101865785 -> 4385.200386104066
inputs/large141.in: cost 6102.530954911454 -> 5665.178712059349
inputs/small114.in: cost 386745.3688118974 -> 386420.3688118974
inputs/small178.in: cost 4295.54

In [2]:
def get_targets(file):
    f = open(file, 'r')
    lines = f.readlines()
    targets = []
    for line in lines:
        name, num, k = line.strip().split()
        targets.append((name, float(num), int(k)))
    return targets
targets = get_targets('target_soln.txt')

In [14]:
# import library

# Request to website and download HTML contents
target_scores = []
for filename, size, num in tqdm(low_scores_all):
    url='https://170-leaderboard.vercel.app/input/'+size+'/'+ num
    req=requests.get(url)
    content=req.text
    soup=BeautifulSoup(content)
    raw = soup.findAll('td')[2].text
    target = float(raw)+0.05
    target_scores.append([filename,target])

100%|██████████| 559/559 [02:48<00:00,  3.32it/s]


In [28]:
target_scores = [[name,score] for name,score in target_scores.items()]

In [16]:

df = pd.read_csv('best_scores.csv')
    
for i in range(len(target_scores)):
    filename, target = target_scores[i][:2]
    approx = 2*np.log(target/100)
    k = math.floor(approx)
    r = df.index[df['name'] == filename]
    num_p = int(df.iloc[r]['k'])
    if abs(k-num_p) > 3:
        k = num_p
    target_scores[i] = [filename, str(target),str(k)]

In [18]:
f = open("target_soln_new.txt", "w")
for line in target_scores:
    f.write(line[0] + ' ' + line[1] + ' ' + line[2])
    f.write('\n')
f.close()


In [20]:
targets = []
def get_targets(file):
    f = open(file, 'r')
    lines = f.readlines()
    targets = []
    for line in lines:
        name, num, k = line.strip().split()
        targets.append((name, float(num), int(k)))
    return targets
targets = get_targets('target_soln.txt')
